# 线性规划 Linear Programming
参考自《Python与运筹优化》

## 逻辑的数学表示
### 条件约束
$ B>0 \space if \space A>0$ 等价于
$$
s.t.
    \begin{cases}
    s \cdot A - max(A, 0) = 0 \\
    s \cdot M + B > 0 \\
    s = 0 \space or \space 1
    \end{cases}
$$ 
其中M为一个充分大的数

- 或逻辑：

$ B>0 \space if \space A_1>0 \space or \space A_2>0$ 等价于
$$
s.t.
    \begin{cases}
    s_1 \cdot A_1 - max(A, 0) = 0 \\
    s_2 \cdot A_2 - max(A, 0) = 0 \\
    (s_1 + s_2) \cdot M + B > 0 \\
    s_1, s_2 = 0 \space or \space 1
    \end{cases}
$$ 

- 与逻辑：

$ B>0 \space if \space A_1>0 \space and \space A_2>0$ 等价于
$$
s.t.
    \begin{cases}
    s_1 \cdot A_1 - max(A, 0) = 0 \\
    s_2 \cdot A_2 - max(A, 0) = 0 \\
    (|s_1 + s_2| - |s_1 - s_2|) \cdot M + B > 0 \\
    s_1, s_2 = 0 \space or \space 1
    \end{cases}
$$ 